<a href="https://colab.research.google.com/github/snpsuen/Deep_Learning_Data/blob/main/script/finetune_gpt2_example03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets

In [2]:
import transformers
print(transformers.__version__)

4.54.0


In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('imdb')
print("type(dataset) = ", type(dataset))
print("dataset.keys() = ", dataset.keys())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

type(dataset) =  <class 'datasets.dataset_dict.DatasetDict'>
dataset.keys() =  dict_keys(['train', 'test', 'unsupervised'])


In [19]:
from transformers import GPT2Tokenizer

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=False)
print("type(tokenized_datasets) = ", type(tokenized_datasets))
print("tokenized_datasets.keys() = ", tokenized_datasets.keys())

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

type(tokenized_datasets) =  <class 'datasets.dataset_dict.DatasetDict'>
tokenized_datasets.keys() =  dict_keys(['train', 'test', 'unsupervised'])


In [20]:
from transformers import GPT2LMHeadModel

# Load the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    max_steps=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

In [22]:
from transformers import Trainer, DataCollatorForLanguageModeling

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)

In [23]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=20, training_loss=3.8749767303466798, metrics={'train_runtime': 288.061, 'train_samples_per_second': 0.139, 'train_steps_per_second': 0.069, 'total_flos': 10451681280000.0, 'train_loss': 3.8749767303466798, 'epoch': 0.0016})

In [24]:
import torch
from torch.utils.data import DataLoader
import math
from tqdm.auto import tqdm

def custom_eval(model, tokenized_datasets, tokenizer, data_collator, training_args, num_steps):
    """
    Performs a custom evaluation for a limited number of steps.

    Args:
        model: The model to evaluate.
        tokenized_datasets: The tokenized dataset dictionary.
        tokenizer: The tokenizer used for the model.
        data_collator: The data collator for preparing batches.
        training_args: The training arguments containing batch size information.
        num_steps: The number of evaluation steps to perform.

    Returns:
        A dictionary containing the evaluation results (average loss and perplexity).
    """
    # Create a smaller evaluation dataset
    # Ensure we don't request more samples than available in the dataset
    max_samples = len(tokenized_datasets['test'])
    num_samples_for_eval = min(num_steps * training_args.per_device_eval_batch_size, max_samples)
    small_eval_dataset = tokenized_datasets['test'].select(range(num_samples_for_eval))

    # Remove the 'text' column as it's not needed for evaluation after tokenization
    small_eval_dataset = small_eval_dataset.remove_columns(['text'])

    # Create a DataLoader for the smaller evaluation dataset.
    eval_dataloader = DataLoader(
        small_eval_dataset,
        batch_size=training_args.per_device_eval_batch_size,
        collate_fn=data_collator
    )

    # Set the model to evaluation mode.
    model.eval()

    # Initialize variables to accumulate loss and track the number of steps.
    total_eval_loss = 0
    num_eval_steps = 0

    # Get the device from the model
    device = model.device

    print(f"Performing custom evaluation for {num_steps} steps...")

    # Iterate over a limited number of batches from the DataLoader.
    # Use tqdm for a progress bar
    for step, batch in enumerate(tqdm(eval_dataloader, total=num_steps)):
        if step >= num_steps:
            break

        # Move batch to the appropriate device (GPU/CPU).
        batch = {k: v.to(device) for k, v in batch.items()}

        # Perform a forward pass through the model to get the loss.
        with torch.no_grad():
            # Ensure 'labels' are present for loss calculation
            if 'labels' not in batch:
                 batch['labels'] = batch['input_ids']

            outputs = model(**batch)
            loss = outputs.loss

        # Accumulate the loss and increment the step count.
        total_eval_loss += loss.item()
        num_eval_steps += 1

    # After the loop, calculate the average loss and perplexity.
    results = {}
    if num_eval_steps > 0:
        average_eval_loss = total_eval_loss / num_eval_steps
        perplexity = math.exp(average_eval_loss)

        results['Average Loss'] = average_eval_loss
        results['Perplexity'] = perplexity

        print(f"Custom Evaluation - Average Loss: {average_eval_loss:.4f}")
        print(f"Custom Evaluation - Perplexity: {perplexity:.4f}")
    else:
        print("No evaluation steps were performed.")
        results['Average Loss'] = None
        results['Perplexity'] = None

    return results

In [25]:
# Define the number of steps for the custom evaluation
num_steps_for_custom_eval = 20

# Call the custom evaluation function
results = custom_eval(
    model=model,
    tokenized_datasets=tokenized_datasets,
    tokenizer=tokenizer,
    data_collator=data_collator,
    training_args=training_args,
    num_steps=num_steps_for_custom_eval
)

# Print the perplexity from the results
if results and 'Perplexity' in results:
    print(f"Perplexity from custom evaluation: {results['Perplexity']:.4f}")
else:
    print("Custom evaluation did not return perplexity.")

Performing custom evaluation for 20 steps...


  0%|          | 0/20 [00:00<?, ?it/s]

Custom Evaluation - Average Loss: 3.6454
Custom Evaluation - Perplexity: 38.2992
Perplexity from custom evaluation: 38.2992


In [26]:
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json')

In [27]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./fine_tuned_gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_gpt2')

# Create a text generation pipeline
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate text
prompt = "Once upon a time"
generated_text = text_generator(prompt, max_length=100, num_return_sequences=1)
print(generated_text[0]['generated_text'])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Once upon a time the two sides of the room had been silent.

"Miho, I'm sorry."

"I'm sorry. I'm sorry that you have the courage to speak up. I'm sorry for what you've done to me, but I don't want to lose you. I'm just not ready to let go."

"I'm sorry, I understand. I'll go back to sleep. I'll be gone when you're done doing what you need to do."

"Umm, I'm sorry. I'll be sure to find out when I'm back."

"I'm sorry."

"I'm sorry. I'm sorry it's that bad. I'd much rather not be around that person anymore."

"I'm sorry, I'm sorry. I'm sorry. I'm sorry."

"Umm, I know it's not going to be easy. I've been through so much."

"I know it's not going to be easy. I've been through so much."

"Umm, I know it's not going to be easy. I've been through so much."

"I know it's not going to be easy. I've


In [28]:
# --- Interactive Prompt ---
print("\n🎭 FTgpt2 Interactive Mode (word-level) — type 'exit' to quit.")
while True:
    prompt = input("\nYou > ").strip()
    if prompt.lower() in ['exit', 'quit']:
        print("Goodbye!")
        break
    if not prompt:
        continue
    try:
        generated_text = text_generator(prompt, max_length=100, num_return_sequences=1)
        print("\nFTgpt2 >", generated_text[0]['generated_text'])
    except Exception as e:
        print(f"⚠️ Error: {e}")
        continue


🎭 FTgpt2 Interactive Mode (word-level) — type 'exit' to quit.

You > A long long time ago


Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



FTgpt2 > A long long time ago, I had thought it was a good idea to write a post about the best part of the series about the great work of Stephen King.

There are now some who think I'm crazy for suggesting that there's too much of a good thing to say about this series. I'm not. And I know I am.

I'm not suggesting that I'm not going to write about the best part of this series. I'm just suggesting that there are more people who actually read it than there are who would consider it a worthy reading.

I'm not suggesting that the best part of this series is the entire series. I'm not saying that.

I'm just saying that the series is better for it.

It's what it is.

So, for me, this is not a great thing. I'm not saying we should all just go to the movies, go to our grandparents, go to our friends. I'm not saying that we should keep writing the books, go to our kids. I'm not saying that we should just sit around and watch movies. I'm not saying that there are no good things in the books. I

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



FTgpt2 > Anyway you want it to be a good one, but you get to the point where you just want to give up and just want to make the game feel like it's a game. Which is where it is, and you've got to find a way to make it that way.

So how does Destiny's in-game economy work?

You can see it in the game world map. You can see the world map and it's huge. Just like on PS2, you get to see the world map. The world map is in the game world map. So you can see that you can see what you're getting into. It's a big part of the game.

It's a big part of the game. You can see it in the world map and it's huge. Just like on PS2, you can see that you can see what you're getting into. It's a big part of the game. Every time you run up against a massive random enemy, you're going to get a new reward. It's a lot of fun.

You're going to get something that you're really good at killing.

Yes, and it's going to be a lot of fun. It's going to be a lot of fun.

You > exit
Goodbye!


In [ ]:
print(tokenized_datasets.keys())

dict_keys(['train'])


In [6]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', 